
# Population analysis

Not runnable on its own. Addition to `run_pest_model.ipynb`. Takes hours to complete. Since it is not so easy to adapt to other problems, it is only here for transparency.

In [ ]:
population_days = pd.read_feather("/content/gdrive/MyDrive/data/ap/population_days_2022.feather")

In [ ]:
len(population_days.loc[population_days.date > 1450].user_id.unique())

8925

In [ ]:
X_pop = torch.from_numpy(data_preprocessor.get_X_numpy(population_days)).float().contiguous().cuda()

### Create ground truth presence

In [ ]:
torch.cuda.empty_cache()

In [ ]:
1825/365

5.0

In [ ]:
len(test_X[:,2])

1487

### Train ground truth model

In [ ]:
#obs_X = torch.vstack((train_X[-7000:, :], test_X))
#obs_y = torch.cat((train_y[-7000:], test_y))

# fit model on the data for the last year (and a bit before)
truth_model, truth_likelihood, _ = train_model(test_X, test_y, model, likelihood, optimizer)

dates = np.arange(1461-10, 1461+365)

with torch.no_grad():
    generated_true_presence = truth_likelihood(truth_model(X_pop))
    generated_true_presence.probs *= scaling_factor * 1.2

	Iter 10/ 50 - Loss: 0.401
	Iter 20/ 50 - Loss: 0.397
	Iter 30/ 50 - Loss: 0.397
	Iter 40/ 50 - Loss: 0.397
	Iter 50/ 50 - Loss: 0.397


In [ ]:
with torch.no_grad():
    generated_true_presence_sample = generated_true_presence.sample()

In [ ]:
generated_true_presence.mean.mean()

tensor(0.1064, device='cuda:0')

In [ ]:
generated_true_presence_sample.mean()

tensor(0.1064, device='cuda:0')

### Run Monte Carlo

In [ ]:
# MONTE CARLO
for j in tqdm(range(6, 100)):
    torch.cuda.empty_cache()
    model, likelihood, optimizer = train_model(train_X, train_y)

    with torch.no_grad():
        generated_true_presence_sample = generated_true_presence.sample()

    population_days["generated_true_presence"] = generated_true_presence_sample.cpu().numpy()
    #population_days.drop(columns=["level_0"]).reset_index().to_feather("/content/gdrive/MyDrive/data/ap/population_days_generated_presence.feather")
    torch.cuda.empty_cache()


    # SCENARIO 0
    scenario0_df = pd.DataFrame()

    for date in dates:
        #print(destandardize_date(date).strftime("%Y-%m-%d"))
        temp_dict = dict()
        population_day = population_days.loc[population_days.date == date]
        temp_dict["monitorings"] = 0
        temp_dict["crop_losses"] = population_day["generated_true_presence"].sum()
        temp_dict["treatments"] = 0
        temp_dict["treatments_directly"] = 0

        scenario0_df = pd.concat([scenario0_df, pd.DataFrame(temp_dict, index=[date])])

    scenario0_df.reset_index().to_feather(f"/content/gdrive/MyDrive/data/ap/scenarios/scenario0_{j}_tuta.feather")

    scenario1_df = pd.DataFrame()

    # SCENARIO 1
    population_days_sc = population_days.copy()
    population_days_sc["treated"] = False
    for date in dates:
        #print(destandardize_date(date).strftime("%Y-%m-%d"))
        temp_dict = dict()
        population_day = population_days_sc.loc[population_days_sc.date == date]
        temp_dict["monitorings"] = len(population_day.loc[(~population_day["treated"])])
        treated = population_day.loc[(~population_day["treated"]) & (population_day["generated_true_presence"])]
        temp_dict["crop_losses"] = 0#len(treated)
        temp_dict["treatments"] = len(treated)
        temp_dict["treatments_directly"] = 0

        treated_days = population_days_sc["user_id"].isin(treated.user_id) & (population_days_sc["date"]>=date) & (population_days_sc["date"]<date+10)
        population_days_sc.loc[treated_days, "treated"] = True
        scenario1_df = pd.concat([scenario1_df, pd.DataFrame(temp_dict, index=[date])])

    scenario1_df.reset_index().to_feather(f"/content/gdrive/MyDrive/data/ap/scenarios/scenario1_{j}_tuta.feather")

    # SCENARIO 2
    scenario2_df = pd.DataFrame()
    for i, date in enumerate(dates):
        #print(destandardize_date(date).strftime("%Y-%m-%d"))
        temp_dict = dict()
        population_day = population_days.loc[population_days.date == date]
        temp_dict["monitorings"] = 0
        temp_dict["crop_losses"] = 0
        temp_dict["treatments"] = (population_day.crop_age % 10 == 0).sum() #(i%10 == 0) * len(population_day)#(population_day.crop_age > 28).sum()
        temp_dict["treatments_directly"] = temp_dict["treatments"]
        scenario2_df = pd.concat([scenario2_df, pd.DataFrame(temp_dict, index=[date])])

    scenario2_df.reset_index().to_feather(f"/content/gdrive/MyDrive/data/ap/scenarios/scenario2_{j}_tuta.feather")

    # SCENARIO 4
    scenario4_df = pd.DataFrame()

    population_days_sc = population_days.copy()
    population_days_sc["presence"] = population_days_sc["generated_true_presence"]
    population_days_sc["monitored"] = False
    population_days_sc["treated"] = False

    risks = np.zeros((8925, len(dates)))
    risks.fill(np.nan)

    for i, date in tqdm(enumerate(dates)):
        # print(destandardize_date(date).strftime("%Y-%m-%d"))
        #test_day = day / 365

        #monitored_samples = population_days_sc.loc[population_days_sc["monitored"]]
        #monitoring_X = torch.from_numpy(data_preprocessor.get_X_numpy(monitored_samples)).float().contiguous().cuda()
        #monitoring_y = torch.from_numpy(data_preprocessor.get_Y_numpy(monitored_samples)).float().contiguous().cuda()

        #obs_X = torch.vstack((train_X, monitoring_X))[-50000:, :]
        #obs_y = torch.cat((train_y, monitoring_y))[-50000:]
        #prediction_model, prediction_likelihood, _ = train_model(obs_X, obs_y, model, likelihood, optimizer)

        population_day = population_days_sc.loc[population_days_sc.date == date]
        X_pop = torch.from_numpy(data_preprocessor.get_X_numpy(population_day)).float().contiguous().cuda()
        with torch.no_grad():
            pred_pop = calibrate(likelihood(model(X_pop)).mean.cpu().numpy()) * scaling_factor

        risks[:len(pred_pop), i] = pred_pop

        # calculate expected utility for decision making
        e_u_inaction = e_u_gamma(pred_pop, 0)
        e_u_monitoring = e_u_gamma(pred_pop, 1)
        e_u_treatment = e_u_gamma(pred_pop, 2)

        # monitor if more utility than inaction and crop is currently not under treatment
        monitoring_condition = (e_u_monitoring > e_u_inaction) & (e_u_monitoring > e_u_treatment) & (~population_day["treated"])

        # treat if monitored and found presence
        treatment_after_monitoring_condition = (monitoring_condition & population_day["presence"].astype(bool)) & (~population_day["treated"])
        treatment_directly_condition = (((e_u_treatment > e_u_inaction) & (e_u_treatment > e_u_monitoring))) & (~population_day["treated"])
        treatment_condition = treatment_after_monitoring_condition | treatment_directly_condition

        # set respective crops to treated for 10 days
        treated_days = population_days_sc["user_id"].isin(population_day.loc[treatment_condition].user_id) & (population_days_sc["date"]>=date) & (population_days_sc["date"]<date+10)
        population_days_sc.loc[treated_days, "treated"] = True

        # calculate loss for crops that are still untreated
        loss_condition = population_day["presence"].astype(bool) & (~population_days_sc.loc[population_days_sc.date == date]["treated"])

        # set monitored rows to monitored to include them in the future prediction model
        population_days_sc.loc[population_days_sc.date == date, "monitored"] = monitoring_condition

        # calculate number of monitorings, losses and treatments
        temp_dict["monitorings"] = monitoring_condition.sum()
        temp_dict["crop_losses"] = loss_condition.sum()
        temp_dict["treatments"] = treatment_condition.sum()
        temp_dict["treatments_directly"] = treatment_directly_condition.sum()


        scenario4_df = pd.concat([scenario4_df, pd.DataFrame(temp_dict, index=[date])])

    np.save(f"/content/gdrive/MyDrive/data/ap/scenarios/risks_scenario4_{j}_tuta.npy", risks)
    scenario4_df.reset_index().to_feather(f"/content/gdrive/MyDrive/data/ap/scenarios/scenario4_{j}_tuta.feather")

    # SCENARIO 3
    scenario3_df = pd.DataFrame()

    population_days_sc = population_days.copy()
    population_days_sc["presence"] = population_days_sc["generated_true_presence"]
    population_days_sc["monitored"] = False
    population_days_sc["treated"] = False

    risks = np.zeros((8925, len(dates)))
    risks.fill(np.nan)

    for i, date in tqdm(enumerate(dates)):
        # print(destandardize_date(date).strftime("%Y-%m-%d"))
        #test_day = day / 365

        monitored_samples = population_days_sc.loc[population_days_sc["monitored"]]
        monitoring_X = torch.from_numpy(data_preprocessor.get_X_numpy(monitored_samples)).float().contiguous().cuda()
        monitoring_y = torch.from_numpy(data_preprocessor.get_Y_numpy(monitored_samples)).float().contiguous().cuda()

        obs_X = torch.vstack((train_X[-7000:, :], monitoring_X[-28000:, :]))
        obs_y = torch.cat((train_y[-7000:], monitoring_y[-28000:]))
        prediction_model, prediction_likelihood, _ = train_model(obs_X, obs_y, model, likelihood, optimizer)

        population_day = population_days_sc.loc[population_days_sc.date == date]
        X_pop = torch.from_numpy(data_preprocessor.get_X_numpy(population_day)).float().contiguous().cuda()
        with torch.no_grad():
            pred_pop = calibrate(prediction_likelihood(prediction_model(X_pop)).mean.cpu().numpy()) * scaling_factor

        risks[:len(pred_pop), i] = pred_pop

        # calculate expected utility for decision making
        e_u_inaction = e_u_gamma(pred_pop, 0)
        e_u_monitoring = e_u_gamma(pred_pop, 1)
        e_u_treatment = e_u_gamma(pred_pop, 2)

        # monitor if more utility than inaction and crop is currently not under treatment
        monitoring_condition = (e_u_monitoring > e_u_inaction) & (e_u_monitoring > e_u_treatment) & (~population_day["treated"])

        # treat if monitored and found presence
        treatment_after_monitoring_condition = (monitoring_condition & population_day["presence"].astype(bool)) & (~population_day["treated"])
        treatment_directly_condition = (((e_u_treatment > e_u_inaction) & (e_u_treatment > e_u_monitoring))) & (~population_day["treated"])
        treatment_condition = treatment_after_monitoring_condition | treatment_directly_condition

        # set respective crops to treated for 10 days
        treated_days = population_days_sc["user_id"].isin(population_day.loc[treatment_condition].user_id) & (population_days_sc["date"]>=date) & (population_days_sc["date"]<date+10)
        population_days_sc.loc[treated_days, "treated"] = True

        # calculate loss for crops that are still untreated
        loss_condition = population_day["presence"].astype(bool) & (~population_days_sc.loc[population_days_sc.date == date]["treated"])

        # set monitored rows to monitored to include them in the future prediction model
        population_days_sc.loc[population_days_sc.date == date, "monitored"] = monitoring_condition

        # calculate number of monitorings, losses and treatments
        temp_dict["monitorings"] = monitoring_condition.sum()
        temp_dict["crop_losses"] = loss_condition.sum()
        temp_dict["treatments"] = treatment_condition.sum()
        temp_dict["treatments_directly"] = treatment_directly_condition.sum()


        scenario3_df = pd.concat([scenario3_df, pd.DataFrame(temp_dict, index=[date])])

    np.save(f"/content/gdrive/MyDrive/data/ap/scenarios/risks_scenario3_{j}_tuta.npy", risks)
    scenario3_df.reset_index().to_feather(f"/content/gdrive/MyDrive/data/ap/scenarios/scenario3_{j}_tuta.feather")

  0%|          | 0/94 [00:00<?, ?it/s]

Iter 10/512 - Loss: 0.764
Iter 20/512 - Loss: 0.671
Iter 30/512 - Loss: 0.603
Iter 40/512 - Loss: 0.547
Iter 50/512 - Loss: 0.503
Iter 60/512 - Loss: 0.462
Iter 70/512 - Loss: 0.423
Iter 80/512 - Loss: 0.392
Iter 90/512 - Loss: 0.369
Iter 100/512 - Loss: 0.351
Iter 110/512 - Loss: 0.337
Iter 120/512 - Loss: 0.326
Iter 130/512 - Loss: 0.319
Iter 140/512 - Loss: 0.312
Iter 150/512 - Loss: 0.307
Iter 160/512 - Loss: 0.303
Iter 170/512 - Loss: 0.3
Iter 180/512 - Loss: 0.297
Iter 190/512 - Loss: 0.294
Iter 200/512 - Loss: 0.291
Iter 210/512 - Loss: 0.289
Iter 220/512 - Loss: 0.288
Iter 230/512 - Loss: 0.286
Iter 240/512 - Loss: 0.284
Iter 250/512 - Loss: 0.283
Iter 260/512 - Loss: 0.282
Iter 270/512 - Loss: 0.28
Iter 280/512 - Loss: 0.28
Iter 290/512 - Loss: 0.278
Iter 300/512 - Loss: 0.277
Iter 310/512 - Loss: 0.277
Iter 320/512 - Loss: 0.276
Iter 330/512 - Loss: 0.275
Iter 340/512 - Loss: 0.274
Iter 350/512 - Loss: 0.274
Iter 360/512 - Loss: 0.272
Iter 370/512 - Loss: 0.271
Iter 380/512 -


0it [00:00, ?it/s]
1it [00:00,  3.05it/s]
2it [00:00,  3.01it/s]
3it [00:00,  3.03it/s]
4it [00:01,  3.04it/s]
5it [00:01,  3.05it/s]
6it [00:01,  3.07it/s]
7it [00:02,  3.07it/s]
8it [00:02,  2.57it/s]
9it [00:03,  2.71it/s]
10it [00:03,  2.82it/s]
11it [00:03,  2.90it/s]
12it [00:04,  2.90it/s]
13it [00:04,  2.89it/s]
14it [00:04,  2.90it/s]
15it [00:05,  2.94it/s]
16it [00:05,  2.84it/s]
17it [00:05,  2.89it/s]
18it [00:06,  2.93it/s]
19it [00:06,  2.88it/s]
20it [00:06,  2.87it/s]
21it [00:07,  2.85it/s]
22it [00:07,  2.91it/s]
23it [00:08,  2.54it/s]
24it [00:08,  2.70it/s]
25it [00:08,  2.80it/s]
26it [00:09,  2.89it/s]
27it [00:09,  2.94it/s]
28it [00:09,  2.98it/s]
29it [00:10,  3.00it/s]
30it [00:10,  3.02it/s]
31it [00:10,  3.03it/s]
32it [00:11,  3.01it/s]
33it [00:11,  3.01it/s]
34it [00:11,  3.00it/s]
35it [00:12,  2.98it/s]
36it [00:12,  2.96it/s]
37it [00:12,  2.94it/s]
38it [00:13,  2.56it/s]
39it [00:13,  2.66it/s]
40it [00:13,  2.74it/s]
41it [00:14,  2.82it/s]
42it 

	Iter 10/ 50 - Loss: 0.279
	Iter 20/ 50 - Loss: 0.267
	Iter 30/ 50 - Loss: 0.264
	Iter 40/ 50 - Loss: 0.262
	Iter 50/ 50 - Loss: 0.262



1it [00:02,  2.33s/it]

...


192it [08:06,  2.72s/it]

	Iter 10/ 50 - Loss: 0.357
	Iter 20/ 50 - Loss: 0.343
	Iter 30/ 50 - Loss: 0.335
	Iter 40/ 50 - Loss: 0.329
	Iter 50/ 50 - Loss: 0.325



193it [08:08,  2.70s/it]

	Iter 10/ 50 - Loss: 0.384
	Iter 20/ 50 - Loss: 0.364
	Iter 30/ 50 - Loss: 0.351
	Iter 40/ 50 - Loss: 0.342
	Iter 50/ 50 - Loss: 0.335



194it [08:11,  2.69s/it]